<a href="https://colab.research.google.com/github/NeerajP2003/118S--Agent-AI/blob/main/Coding_Exercise_Prompt_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# customer_support_chain.py
from typing import Dict, Any, List
import json

class FakeLLM:
    """Tiny rule-based 'LLM' so this runs without external APIs."""
    def intent(self, message: str) -> Dict[str, Any]:
        msg = message.lower()
        if any(w in msg for w in ["refund", "charged", "billing", "invoice", "payment"]):
            return {"intent": "billing", "confidence": 0.88}
        if any(w in msg for w in ["error","bug","crash","not working","failed","issue"]):
            return {"intent": "technical_issue", "confidence": 0.86}
        if any(w in msg for w in ["login","password","otp","2fa","locked","access"]):
            return {"intent": "account_access", "confidence": 0.84}
        return {"intent": "other", "confidence": 0.55}

    def gather(self, intent: str) -> List[str]:
        questions = {
            "billing": [
                "Which invoice or charge are you referring to (date/amount)?",
                "What is the payment method (card/UPI/netbanking)?"
            ],
            "technical_issue": [
                "What action were you performing when the issue occurred?",
                "Can you share the exact error message or a screenshot?",
                "What device/OS/app version are you using?"
            ],
            "account_access": [
                "What happens when you try to sign in (error text)?",
                "Have you tried password reset or alternative 2FA methods?"
            ],
            "other": [
                "Could you describe your request in one line?"
            ]
        }
        return questions.get(intent, questions["other"])

    def pick_solution(self, intent: str, facts: Dict[str, str], kb: Dict[str, str]) -> str:
        return kb.get(intent, "Please provide more details so I can assist.")

    def draft_reply(self, solution: str) -> str:
        steps = [s.strip() for s in solution.split('.') if s.strip()]
        lines = ["Thanks for reaching out — I think I understand the issue. Let's try this:"]
        for i, s in enumerate(steps, 1):
            lines.append(f"{i}. {s}.")
        lines.append("If that doesn’t resolve it, I’m here to help with next steps.")
        return "\n".join(lines)

# --- Mini knowledge base (replace with your real KB) ---
KB = {
    "billing": (
        "Open Billing > Transactions > Select the charge > Click 'Dispute'. "
        "Attach a brief note and submit. You'll get an email within 24 hours."
    ),
    "technical_issue": (
        "Restart the app. Clear cache in Settings. If the error persists, reinstall the app. "
        "Share logs from Settings > Diagnostics."
    ),
    "account_access": (
        "Use 'Forgot password'. If 2FA fails, choose 'Use backup codes'. "
        "Still stuck? Contact support to verify identity."
    )
}

def run_chain(user_message: str, extra_facts: Dict[str, str] | None = None) -> None:
    llm = FakeLLM()
    facts = extra_facts or {}

    # 1) Intent Classification
    ic = llm.intent(user_message)
    intent = ic["intent"]

    # 2) Info Gathering
    questions = llm.gather(intent)

    # 3) Solution Retrieval
    solution = llm.pick_solution(intent, facts, KB)

    # 4) Draft Reply
    reply = llm.draft_reply(solution)

    # --- Show chain outputs ---
    print("User message:", user_message)
    print("\n[1] Intent Classification:", json.dumps(ic, indent=2))
    print("\n[2] Info Gathering Questions:")
    for q in questions:
        print("-", q)
    print("\n[3] Chosen Solution Snippet:\n", solution)
    print("\n[4] Final Support Reply:\n", reply)

if __name__ == "__main__":
    run_chain(
        "Hi, I was charged twice yesterday and want a refund.",
        {"charge_date": "2025-10-01", "payment_method": "card"}
    )

User message: Hi, I was charged twice yesterday and want a refund.

[1] Intent Classification: {
  "intent": "billing",
  "confidence": 0.88
}

[2] Info Gathering Questions:
- Which invoice or charge are you referring to (date/amount)?
- What is the payment method (card/UPI/netbanking)?

[3] Chosen Solution Snippet:
 Open Billing > Transactions > Select the charge > Click 'Dispute'. Attach a brief note and submit. You'll get an email within 24 hours.

[4] Final Support Reply:
 Thanks for reaching out — I think I understand the issue. Let's try this:
1. Open Billing > Transactions > Select the charge > Click 'Dispute'.
2. Attach a brief note and submit.
3. You'll get an email within 24 hours.
If that doesn’t resolve it, I’m here to help with next steps.
